In [2]:
import numpy as onp
import jax.numpy as np
from jax import random
from jax import jit
from jax import vmap
from jax import grad
from jax import lax
from jax import config
config.update("jax_enable_x64", True)


from jax_md import space, smap, energy, minimize, quantity, simulate, partition, util
from jax_md.util import f32

from collections import namedtuple

vectorize = np.vectorize

# normalize = lambda v: v / np.linalg.norm(v, axis=1, keepdims=True)

def normalize(v, v_lim=0):
    v_norm = np.linalg.norm(v, axis=1, keepdims=True)
    return np.where(v_norm > v_lim, v/v_norm, v)


from functools import partial
from simulator.utils import angle_correct, normal, general_force_generator_TEST_1, general_force_generator_TEST_2, general_force_generator_TEST_3, general_force_generator_TEST_4
from simulator.render import render
from simulator.utils import time_to_collide, ttc_tot, ttc_potential_fn, wall_energy, normalize_cap
from simulator.force import goal_velocity_force_tot
import matplotlib.pyplot as plt

class PedestrianState(namedtuple("PedestrianState", ['position', 'velocity', 'radius', 'goal_velocity'])):
    # 'group_velocity'
    def orientation(self):
        """Returns the orientation angle of all pedestrians"""
        return angle_correct(np.atan2(self.position[:, 0], self.position[:, 1]))

In [ ]:
displacement, shift = space.free()

dpos = np.array([5, 0])
rela_v = np.array([4.01, 2.99]) / 5
print(time_to_collide(dpos, rela_v, np.zeros((2, ), dtype=f32), 1.5))

In [ ]:
np.sqrt(-1)

In [ ]:
pos = np.array([[0.0, 0.0],
                [3.0, 0.0]])

velocity = np.array([[1.0, 0.0],
                     [0.0, 0.0]])

print(ttc_tot(pos, velocity, 1, displacement))

print(ttc_potential_fn(1.5, np.reshape(ttc_tot(pos, velocity, 1, displacement), shape=(4, )), 3.0))

In [ ]:
dt = np.linspace(0.2, 3., 60)
Es = ttc_potential_fn(1.5, dt, 3.0)

plt.plot(dt, Es)

In [ ]:
np.arange(0, 10)


In [ ]:
pos = np.array([[0.0, 0.0],
                [3.0, 0.0]])

velocity = np.array([[1.0, 0.0],
                     [0.0, 0.0]])

print(ttc_force_tot(pos, velocity, 1.0, displacement))
print(normalize_cap(ttc_force_tot(pos, velocity, 1.0, displacement), 5))

print(np.sum(ttc_force_tot(pos, velocity, 1.0, displacement), axis=1))
print(np.sum(normalize_cap(ttc_force_tot(pos, velocity, 1.0, displacement)), axis=1))

# print(normalize(ttc_force_tot(pos, velocity, 1.0, displacement)))
# print(np.sum(ttc_force_tot(pos, velocity, 1.0, displacement), axis=1))
# print(np.sum(normalize(ttc_force_tot(pos, velocity, 1.0, displacement)), axis=1))

In [ ]:
# @jit
# def wall_energy(pos, wall):
#    wall_start, wall_end = wall
#    wall_tangent = wall_end - wall_start
#    wall_normal = test_normalize_cap(np.array([-wall_tangent[1], wall_tangent[0]]))

#    return 1/np.dot(pos, wall_normal)

print(wall_energy(np.array([[0.0, 5.0], [3.0, 10.0]]), np.array([[0, 0], [10, 0]])))
print(quantity.force(wall_energy)(np.array([[0.0, 5.0], [3.0, 10.0]]), np.array([[0, 0], [10, 0]])))

In [ ]:
def wall_dist(pos, start, end):
    wall_len = np.dot(end - start, end - start)
    # t = max(0, min(1, np.dot(pos - start, end - start) / wall_len))
    t = np.max(np.array([0, np.min(np.array([1, np.dot(pos - start, end - start) / wall_len]), axis = 0)]), axis = 0)
    proj = start + t * (end - start)
    return displacement(pos, proj)

print(wall_dist(np.array([0.0, 3.0]), np.array([4.0, 0.0]), np.array([10.0, 0.0])))

In [ ]:
class StraightWall(namedtuple('StraightWall', ['start', 'end'])):
    pass

pos = np.array([[1.0, 0.0],
                [3.0, 0.0]])

wall1 = StraightWall(np.array([4.0, 0.0]), np.array([10.0, 0.0]))
wall2 = StraightWall(np.array([0.0, 1.0]), np.array([0.0, 9.0]))
walls = np.array([wall1, wall2])

def true_wall_energy_tot(poss, walls, displacement):
    return vmap(wall_energy_tot, (None, 0, None))(poss, walls, displacement)

true_wall_energy_tot(pos, walls, displacement)


In [2]:
paral = np.array([
    [[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
])
perpen = np.array([
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
])

force1 = general_force_generator_TEST_1(paral, perpen, 10., 10.)
force2 = general_force_generator_TEST_2(paral, perpen, 10., 10.)
force3 = general_force_generator_TEST_3(paral, perpen, 10., 10.)
force4 = general_force_generator_TEST_4(paral, perpen, 10., 10.)

test_1_init_runtime = 9.5367431640625e-07
test_2_init_runtime = 0.10322093963623047
test_3_init_runtime = 9.5367431640625e-07
test_4_init_runtime = 2.1457672119140625e-06


In [7]:
dpos = np.array([10.0, 2.0])
V_1 = np.array([1.0, 0.0])
V_2 = np.array([2.0, 0.0])
print(force1(dpos, V_1, V_2))
print(force2(dpos, V_1, V_2))
print(force3(dpos, V_1, V_2))
print(force4(dpos, V_1, V_2))

test_1_runtime = 0.044194936752319336
[-0.57126507  0.        ]
test_2_runtime = 0.042388916015625
[-0.57126507  0.        ]
test_3_runtime = 0.31406497955322266
[-0.57126507  0.        ]
test_4_runtime = 0.3099660873413086
[-0.57126507  0.        ]


In [6]:
np.round(1.1010, 0)

Array(1., dtype=float64, weak_type=True)

In [6]:
a = np.array([1, 2, 3, 4, 5, 6])
A = np.tensordot(np.tensordot(a, a, axes=0), a, axes=0)

print(A)

[[[  1   2   3   4   5   6]
  [  2   4   6   8  10  12]
  [  3   6   9  12  15  18]
  [  4   8  12  16  20  24]
  [  5  10  15  20  25  30]
  [  6  12  18  24  30  36]]

 [[  2   4   6   8  10  12]
  [  4   8  12  16  20  24]
  [  6  12  18  24  30  36]
  [  8  16  24  32  40  48]
  [ 10  20  30  40  50  60]
  [ 12  24  36  48  60  72]]

 [[  3   6   9  12  15  18]
  [  6  12  18  24  30  36]
  [  9  18  27  36  45  54]
  [ 12  24  36  48  60  72]
  [ 15  30  45  60  75  90]
  [ 18  36  54  72  90 108]]

 [[  4   8  12  16  20  24]
  [  8  16  24  32  40  48]
  [ 12  24  36  48  60  72]
  [ 16  32  48  64  80  96]
  [ 20  40  60  80 100 120]
  [ 24  48  72  96 120 144]]

 [[  5  10  15  20  25  30]
  [ 10  20  30  40  50  60]
  [ 15  30  45  60  75  90]
  [ 20  40  60  80 100 120]
  [ 25  50  75 100 125 150]
  [ 30  60  90 120 150 180]]

 [[  6  12  18  24  30  36]
  [ 12  24  36  48  60  72]
  [ 18  36  54  72  90 108]
  [ 24  48  72  96 120 144]
  [ 30  60  90 120 150 180]
  [ 36  72

In [11]:
saver = []

@jit
def fn(i, x):
    ans = i + x
    saver.append(ans)
    return ans

fin_ans = 0
fin_ans = fn(69, fin_ans)
fin_ans = fn(420, fin_ans)

# fin_ans = lax.fori_loop(0, 10, fn, 0)

print(saver)
print(fin_ans)

[Traced<~int64[]>with<DynamicJaxprTrace>]
489


In [12]:
np.zeros(7)

Array([0., 0., 0., 0., 0., 0., 0.], dtype=float64)

In [14]:
np.where(False, np.array([1, 2, 3]), np.array([0, 0, 0]))

Array([0, 0, 0], dtype=int64)

In [5]:
def fn(*vars):
    return vars

fn(1, 2, 3)

(1, 2, 3)

In [8]:
A = {'a' : 3, 'b' : 2, 'c' : 5}
a, b, c = A.values()
print(a, b, c)

3 2 5


In [4]:
def fn(vec_1, vec_2):
    return vec_1[0] - vec_2[0]

print(grad(fn)(np.array([1., 69.]), np.array([1., 420.])))

[1. 0.]


In [10]:
def fn(idx, x):
    return idx + 1, x

lax.scan(fn, 0, length=-2)

(Array(0, dtype=int64, weak_type=True), None)

In [4]:
np.arange(0, 10)

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [4]:
vel = np.array([[10., 0], [0.5, 0]])
# goal_speed = 1.
goal_speed = np.array([1., 2.])
goal_orientation = np.array([0., onp.pi / 2])
print(goal_velocity_force_tot(vel, goal_speed, goal_orientation))

[[-18.   0.]
 [ -1.   4.]]


In [24]:
A = np.array(
    [[0, 1],
     [1, 2],
     [2, 3]]
)
B = np.array(
    [0.5, 3, 7]
)
B = np.stack([B, B], axis=1)
print(B)
A / B


[[0.5 0.5]
 [3.  3. ]
 [7.  7. ]]


Array([[0.        , 2.        ],
       [0.33333333, 0.66666667],
       [0.28571429, 0.42857143]], dtype=float64)

In [12]:
np.array([B] * 10)

Array([[0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ],
       [0.5, 3. ]], dtype=float64)

In [19]:
np.arctan2(np.array([-0.01, 0.5]), np.array([-1, 0.8]))

Array([-3.13159299,  0.55859932], dtype=float64)

In [26]:
goals = np.array([[0, 1], [1, 0]])

@vmap
def cond_fn_0(pos, goal):
    disp_to_goal = goal - pos
    dist_to_goal = np.linalg.norm(disp_to_goal)
    return np.where(dist_to_goal > 0.1, goals[0], goals[1])

@vmap
def cond_fn_1(pos, goal):
    return goals[1]



pos = np.array([[0, 0],  [0, 1], [1, 0]])
goal = np.array([[0, 0.1], [0.1, 0], [0, 69]])
print(cond_fn_0(pos, goal))
print(cond_fn_1(pos, goal))

cond = vmap(np.array_equal, (0, None))(pos, np.array([0, 0]))
np.stack((cond, cond), axis=1)

[[1 0]
 [0 1]
 [0 1]]
[[1 0]
 [1 0]
 [1 0]]


Array([[ True,  True],
       [False, False],
       [False, False]], dtype=bool)

In [1]:
for i in range(9, -1, -1):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [8]:
np.array([1, 2]) + np.array([[0, 0], [1, 1], [2, 2]])

Array([[1, 2],
       [2, 3],
       [3, 4]], dtype=int64)

In [ ]:
key = random.PRNGKey(69)
random.choice(key, np.array([[0, 1], [2, 3], [4, 5]]), (20,))

Array([[2, 3],
       [2, 3],
       [2, 3],
       [2, 3],
       [4, 5],
       [4, 5],
       [4, 5],
       [2, 3],
       [2, 3],
       [4, 5],
       [4, 5],
       [0, 1],
       [2, 3],
       [4, 5],
       [4, 5],
       [2, 3],
       [4, 5],
       [0, 1],
       [2, 3],
       [4, 5]], dtype=int64)

In [14]:
np.atan2(10, 5)

Array(1.10714872, dtype=float64, weak_type=True)